# Pitch

In [2]:
import penaltyblog as pb
from penaltyblog.matchflow import Flow, get_field, get_statsbomb_x, get_statsbomb_y
import numpy as np
import pandas as pd


In [3]:
flow = (
    Flow.statsbomb.from_github_file(match_id=3788743)
    .filter(lambda e: get_field("type.name")(e) == "Shot")
    .assign(
        player_name=get_field("player.name"),
        position_name=get_field("position.name"),
        team_name=get_field("team.name"),
        xg=get_field("shot.statsbomb_xg"),
        x=get_statsbomb_x(),
        y=get_statsbomb_y(),        
    )
    .select("minute", "player_name", "position_name", "team_name", "x", "y", "xg")
    .sort("minute")
)

df = flow.to_pandas()

df.head(10)


,minute,player_name,position_name,team_name,x,y,xg
0,9,Romelu Lukaku Menama,Center Forward,Belgium,107.6,37.7,0.522065
1,13,Mário Figueira Fernandes,Right Back,Russia,112.2,42.5,0.084538
2,15,Magomed Ozdoev,Left Defensive Midfield,Russia,98.5,41.1,0.051355
3,17,Leander Dendoncker,Right Defensive Midfield,Belgium,103.6,42.8,0.088322
4,21,Thorgan Hazard,Left Wing Back,Belgium,115.0,31.6,0.084841
5,28,Toby Alderweireld,Right Center Back,Belgium,88.6,54.7,0.006016
6,33,Thomas Meunier,Right Wing Back,Belgium,109.2,45.7,0.370858
7,44,Youri Tielemans,Left Defensive Midfield,Belgium,101.1,55.6,0.026723
8,45,Roman Zobnin,Right Wing,Russia,93.1,46.2,0.032959
9,47,Yannick Ferreira Carrasco,Left Attacking Midfield,Belgium,108.8,28.9,0.077180


In [4]:
flow = (
    Flow.statsbomb.from_github_file(3788743)
    .filter(lambda e: get_field("type.name")(e) == "Shot")
    .assign(
        player_name=get_field("player.name"),
        xg=get_field("shot.statsbomb_xg")
    )
    .group_by("player_name")
    .summary(
        shots=("shot", "count"),
        total_xg=("xg", "sum"),
        avg_xg_2=("xg", "mean"),
        avg_xg=lambda records: round(
            sum(r["xg"] for r in records if r["xg"] is not None) / len(records), 3
        )
    )
    .sort("total_xg", reverse=True)
    .limit(5)
    .to_pandas()
)

flow

,player_name,shots,total_xg,avg_xg_2,avg_xg
0,Romelu Lukaku Menama,3,0.739415,0.246472,0.246
1,Thomas Meunier,1,0.370858,0.370858,0.371
2,Leander Dendoncker,1,0.088322,0.088322,0.088
3,Thorgan Hazard,1,0.084841,0.084841,0.085
4,Mário Figueira Fernandes,1,0.084538,0.084538,0.085


In [6]:
(
    Flow.statsbomb.from_github_file(3788743)
    .assign(
        xg=get_field("shot.statsbomb_xg"),       
    )    
    .summary(
      total_shots="count",
      mean_xg=("xg","mean"),
      std_xg=("xg","std"),
    )
    .to_pandas()
)

,total_shots,mean_xg,std_xg
0,3858,0.117902,0.142595


In [12]:
(
    Flow.statsbomb.from_github_file(3788743)
      .filter(lambda e: get_field("type.name")(e) == "Shot")
      .assign(
        player=get_field("player.name"),
        xg=get_field("shot.statsbomb_xg")
      )
      .group_by("player")
      .row_number("xg", new_field="shot_rank", reverse=True)
      .ungroup()
      .select("player", "shot_rank", "xg")
      .to_pandas()
)


,player,shot_rank,xg
0,Romelu Lukaku Menama,1,0.522065
1,Romelu Lukaku Menama,2,0.172413
2,Romelu Lukaku Menama,3,0.044936
3,Mário Figueira Fernandes,1,0.084538
4,Magomed Ozdoev,1,0.051355
5,Leander Dendoncker,1,0.088322
6,Thorgan Hazard,1,0.084841
7,Toby Alderweireld,1,0.006016
8,Thomas Meunier,1,0.370858
9,Youri Tielemans,1,0.026723


In [ ]:
# df = pd.DataFrame({
#     'x': np.random.uniform(0, 100, 11),
#     'y': np.random.uniform(0, 100, 11),
#     'name': [f'Player {i+1}' for i in range(11)]
# })

pitch = pb.pitch.Pitch(width=800, height=400, theme="minimal", provider="statsbomb")
pitch.plot_scatter(df, x='x', y='y', hover='player_name')
pitch.show()


In [ ]:
from penaltyblog.pitch import Pitch

# example: Women’s World Cup final (competition 43, season 3, match 8658)
df = load_statsbomb_events(43, 3)

# filter only shots:
shots = df[df['type.name']=='Shot']

pitch = Pitch(provider="statsbomb", theme="classic", title="Shot map—WWC Final")
pitch.plot_scatter(shots, x='x', y='y',
                   hover='player.name', marker=dict(size=12))
pitch.show()


In [ ]:
df2 = pd.DataFrame({
    'x': np.random.uniform(0, 120, 500),
    'y': np.random.uniform(0, 80, 500)
})

# Create the pitch and plot the heatmap
pitch = pb.pitch.Pitch(theme="night")
pitch.plot_heatmap(df2, x='x', y='y')
pitch.show()

In [ ]:
df2 = pd.DataFrame({
    'x': np.random.uniform(10, 50, 500),
    'y': np.random.uniform(20, 75, 500)
})

# Create the pitch and plot the heatmap
pitch = pb.pitch.Pitch(theme="turf")
pitch.plot_kde(df2, x='x', y='y', bandwidth=0.3,)
pitch.show()

In [ ]:
data = pd.DataFrame({
    'x': [30, 50, 70],
    'y': [40, 60, 20],
    'x2': [45, 65, 80],
    'y2': [50, 70, 30],
    'player': ['Alice', 'Bob', 'Carol']
})

# Create a pitch with a dark theme and Opta dimensions
pitch = pb.pitch.Pitch(theme="night")

# Plot the passes as arrows
pitch.plot_arrows(
    data, x='x', y='y', x_end='x2', y_end='y2'
)

# Optional: plot player positions at the start of the arrows
pitch.plot_scatter(data, x='x', y='y', hover='player')

# Show the pitch
pitch.show()


In [ ]:
df = pd.DataFrame({
    'x': [10, 80, 70],
    'y': [10, 60, 20],
    'x2': [40, 65, 80],
    'y2': [50, 70, 30],
    'player': ['Alice', 'Bob', 'Carol']
})

pitch = pb.pitch.Pitch(width=800, height=600, theme="turf", provider="wyscout",)
pitch.plot_comets(df, x='x', y='y', x_end='x2', y_end='y2', segments=20,hover='player')
pitch.show()

In [ ]:
from 